In [1]:
import numpy as np, gc
import pandas as pd

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/student-performance-and-game-play/sample_submission.csv
/kaggle/input/student-performance-and-game-play/train_labels.csv
/kaggle/input/student-performance-and-game-play/train.csv
/kaggle/input/student-performance-and-game-play/test.csv
/kaggle/input/student-performance-and-game-play/jo_wilder/competition.cpython-37m-x86_64-linux-gnu.so
/kaggle/input/student-performance-and-game-play/jo_wilder/__init__.py


In [3]:
tmp = pd.read_csv("/kaggle/input/student-performance-and-game-play/train.csv",usecols=[0])
tmp = tmp.groupby('session_id').session_id.agg('count')
PIECES = 10
CHUNK = int( np.ceil(len(tmp)/PIECES) )
reads = []
skips = [0]
for k in range(PIECES):
    a = k*CHUNK
    b = (k+1)*CHUNK
    if b>len(tmp): b=len(tmp)
    r = tmp.iloc[a:b].sum()
    reads.append(r)
    skips.append(skips[-1]+r)

In [4]:
train = pd.read_csv("/kaggle/input/student-performance-and-game-play/train.csv", nrows=reads[0])
train_label = pd.read_csv("/kaggle/input/student-performance-and-game-play/train_labels.csv")
train_label['session'] = train_label.session_id.apply(lambda x: int(x.split('_')[0]) )
train_label['q'] = train_label.session_id.apply(lambda x: int(x.split('_')[-1][1:]) )
categorical_features = ['event_name', 'fqid', 'room_fqid', 'text']
numerical_features = ['elapsed_time','level','page','room_coor_x', 'room_coor_y', 
        'screen_coor_x', 'screen_coor_y', 'hover_duration']
events = ['navigate_click','person_click','cutscene_click','object_click',
          'map_hover','notification_click','map_click','observation_click',
          'checkpoint']

In [5]:
def feature_engineer(train):
    dfs = []
    for c in categorical_features:
        tmp = train.groupby(['session_id','level_group'])[c].agg('nunique')
        tmp.name = tmp.name + '_nunique'
        dfs.append(tmp)
    for c in numerical_features:
        tmp = train.groupby(['session_id','level_group'])[c].agg('mean')
        tmp.name = tmp.name + '_mean'
        dfs.append(tmp)
    for c in numerical_features:
        tmp = train.groupby(['session_id','level_group'])[c].agg('std')
        tmp.name = tmp.name + '_std'
        dfs.append(tmp)
    for c in events: 
        train[c] = (train.event_name == c).astype('int8')
    for c in events + ['elapsed_time']:
        tmp = train.groupby(['session_id','level_group'])[c].agg('sum')
        tmp.name = tmp.name + '_sum'
        dfs.append(tmp)
    train = train.drop(events,axis=1)
        
    df = pd.concat(dfs,axis=1)
    df = df.fillna(-1)
    df = df.reset_index()
    df = df.set_index('session_id')
    return df


In [6]:
%%time
all_pieces = []
for k in range(PIECES):
    SKIPS = 0
    if k>0: SKIPS = range(1,skips[k]+1)
    train = pd.read_csv('/kaggle/input/student-performance-and-game-play/train.csv',nrows=reads[k], skiprows=SKIPS)
    df_tr = feature_engineer(train)
    all_pieces.append(df_tr)

del train; gc.collect()
df_tr = pd.concat(all_pieces, axis=0)

CPU times: user 7min 8s, sys: 19 s, total: 7min 27s
Wall time: 7min 59s


In [7]:
from sklearn.model_selection import cross_val_score, train_test_split, KFold, GroupKFold
from sklearn.metrics import f1_score

# Define features 
FEATURES = [c for c in df_tr.columns if c != 'level_group']
ALL_USERS = df_tr.index.unique()
print('Train with', len(FEATURES) ,'features and ', len(ALL_USERS) ,'users info')

Train with 30 features and  23562 users info


In [8]:
from sklearn.model_selection import KFold, GroupKFold
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
gkf = GroupKFold(n_splits=5)
oof = pd.DataFrame(data=np.zeros((len(ALL_USERS),18)), index=ALL_USERS)
models = {}
for i, (train_index, test_index) in enumerate(gkf.split(X=df_tr, groups=df_tr.index)):
    xgb_params = {
    'objective' : 'binary:logistic','eval_metric':'logloss','learning_rate': 0.05,'max_depth': 4,'n_estimators': 1000,'early_stopping_rounds': 50,'tree_method':'hist','subsample':0.8,'colsample_bytree': 0.4,'use_label_encoder' : False}
    for t in range(1,19):
        if t<=3: grp = '0-4'
        elif t<=13: grp = '5-12'
        elif t<=22: grp = '13-22'
        train_x = df_tr.iloc[train_index]
        train_x = train_x.loc[train_x.level_group == grp]
        train_users = train_x.index.values
        train_y = train_label.loc[train_label.q==t].set_index('session').loc[train_users]
        valid_x = df_tr.iloc[test_index]
        valid_x = valid_x.loc[valid_x.level_group == grp]
        valid_users = valid_x.index.values
        valid_y = train_label.loc[train_label.q==t].set_index('session').loc[valid_users]
        clf =  XGBClassifier(**xgb_params)
        clf.fit(train_x[FEATURES].astype('float32'), train_y['correct'],eval_set=[ (valid_x[FEATURES].astype('float32'), valid_y['correct']) ],verbose=0)
        models[f'{grp}_{t}'] = clf
        oof.loc[valid_users, t-1] = clf.predict_proba(valid_x[FEATURES].astype('float32'))[:,1]
        


In [9]:
# true labels into dataframe
true = oof.copy()
k = 0

while k < 18:
    # GET TRUE LABELS
    tmp = train_label.loc[train_label.q == k+1].set_index('session').loc[ALL_USERS]
    true[k] = tmp.correct.values
    k += 1

#best threshold    
threshold, step, listA, listB, bestS, bestT = 0.4, 0.01, [], [], 0, 0

while threshold <= 0.8:
    print(f'{threshold:.02f}, ', end='')
    preds = (oof.values.reshape((-1)) > threshold).astype('int')
    m = f1_score(true.values.reshape((-1)), preds, average='macro')
    listA.append(m)
    listB.append(threshold)
    if m > bestS:
        bestS = m
        bestT = threshold
    threshold += step
    
m = f1_score(true.values.reshape((-1)), (oof.values.reshape((-1)) > bestT).astype('int'), average='weighted')
print('\nOverall F1 score is', m)

0.40, 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.50, 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.60, 0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.69, 0.70, 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 
Overall F1 score is 0.7315582455030779


In [10]:
import jo_wilder
env = jo_wilder.make_env()
iter_test = env.iter_test()


import gc
del train_label, df_tr, oof, true
_ = gc.collect()

In [11]:
limits = {'0-4':(1,4), '5-12':(4,14), '13-22':(14,19)}
for (test, sample_submission) in iter_test:
    df = feature_engineer(test)
    grp = test.level_group.values[0]
    a,b = limits[grp]
    for t in range(a,b):
        clf = models[f'{grp}_{t}']
        p = clf.predict_proba(df[FEATURES].astype('float32'))[0,1]
        mask = sample_submission.session_id.str.contains(f'q{t}')
        sample_submission.loc[mask,'correct'] = int( p > bestT)    
    env.predict(sample_submission)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


In [12]:
## the end result is a submission file containing all test session predictions
! head submission.csv

session_id,correct
20090109393214576_q1,1
20090109393214576_q2,1
20090109393214576_q3,1
20090109393214576_q4,1
20090109393214576_q5,0
20090109393214576_q6,1
20090109393214576_q7,1
20090109393214576_q8,0
20090109393214576_q9,1
